In [1]:
import sys
sys.path.append("..")
import logging
import random
import neptune
import numpy as np
import syft as sy
from torch import load
from torchvision import transforms
from federated_learning.FLCustomDataset import FLCustomDataset
from federated_learning.FederatedLearning import FederatedLearning
from federated_learning.helper import utils

CONFIG_PATH = '../configs/defaults.yml'

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/ubuntu/.local/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [2]:
arguments = dict()
arguments['--reg'] = 0.0
arguments['--output-prefix'] = "mnist_no_attack"
arguments['--server_model'] = "data_output/20201108_225254_mnist_w0/models/server_model_49"
arguments['--local-log'] = "False"
arguments['--neptune-log'] = "False"
configs = utils.load_config(CONFIG_PATH)
logging.basicConfig(format='%(asctime)s %(message)s', level=configs['log']['level'])
random.seed(configs['runtime']['random_seed'])

# From command line
epochs_num = 1
rounds_num = 1

log_enable = True if arguments['--local-log'] == "True" or \
    arguments['--local-log'] == "true" else False

output_dir = None
if log_enable:
    output_dir = utils.make_output_dir(
        configs['log']['root_output_dir'], arguments['--output-prefix'])

neptune_enable = True if arguments['--neptune-log'] == "True" or \
    arguments['--neptune-log'] == "true" else False

fl = FederatedLearning(
    configs['runtime']['batch_size'], 
    configs['runtime']['test_batch_size'], 
    configs['runtime']['lr'], 
    float(arguments['--reg']) if arguments['--reg'] is not None else 0.0,
    configs['runtime']['momentum'], 
    neptune_enable, log_enable, 
    configs['log']['interval'], 
    output_dir, 
    configs['runtime']['random_seed'])

test_data = utils.load_mnist_data_test(configs['data']['MNIST_PATH'])
test_dataset = utils.get_mnist_dataset(test_data)
test_dataloader = utils.get_mnist_dataloader(
    test_dataset, configs['runtime']['test_batch_size'], shuffle=True)

2020-11-09 00:59:05,820 Initializing Federated Learning class...
2020-11-09 00:59:06,861 Loading test data from MNIST dataset.
2020-11-09 00:59:06,914 Creating MNIST dataset.
2020-11-09 00:59:06,915 Creating MNIST data loader.


In [3]:
server_model = load("../data_output/20201108_225254_mnist_w0/models/server_model_49")
fl.test(server_model, test_dataloader, 1)

2020-11-09 01:00:00,730 Test Average loss: 26.7556, Accuracy: 9761/10000 (98%)



97.61